In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('..')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split
from merf.evaluator import plot_bhat, plot_training_stats

In [24]:
train_df = pd.read_csv('../data/kaggle_train.csv')
test_df = pd.read_csv('../data/kaggle_test.csv')
store_df = pd.read_csv('../data/kaggle_store.csv')

/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# There are 1115 Stores
train_df['Store'].unique()

array([   1,    2,    3, ..., 1113, 1114, 1115])

In [34]:
# These are the store level variates
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# Goal

Ok. So I have to predict the NEXT six weeks out. N+1 all the way to N+(7*6) days. Makes sense. 

In [31]:
train_df[train_df['Store'] == 1]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1115,1,4,2015-07-30,5020,546,1,1,0,1
2230,1,3,2015-07-29,4782,523,1,1,0,1
3345,1,2,2015-07-28,5011,560,1,1,0,1
4460,1,1,2015-07-27,6102,612,1,1,0,1
5575,1,7,2015-07-26,0,0,0,0,0,0
6690,1,6,2015-07-25,4364,500,1,0,0,0
7805,1,5,2015-07-24,3706,459,1,0,0,0
8920,1,4,2015-07-23,3769,503,1,0,0,0
10035,1,3,2015-07-22,3464,463,1,0,0,0


In [30]:
test_df[test_df['Store'] == 1]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
856,857,1,3,2015-09-16,1.0,1,0,0
1712,1713,1,2,2015-09-15,1.0,1,0,0
2568,2569,1,1,2015-09-14,1.0,1,0,0
3424,3425,1,7,2015-09-13,0.0,0,0,0
4280,4281,1,6,2015-09-12,1.0,0,0,0
5136,5137,1,5,2015-09-11,1.0,0,0,0
5992,5993,1,4,2015-09-10,1.0,0,0,0
6848,6849,1,3,2015-09-09,1.0,0,0,0
7704,7705,1,2,2015-09-08,1.0,0,0,0


In [27]:
# I have about three years of data for most stores
train_df.groupby('Store').count()

,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,,,,,,,,
1,942,942,942,942,942,942,942,942
2,942,942,942,942,942,942,942,942
3,942,942,942,942,942,942,942,942
4,942,942,942,942,942,942,942,942
5,942,942,942,942,942,942,942,942
6,942,942,942,942,942,942,942,942
7,942,942,942,942,942,942,942,942
8,942,942,942,942,942,942,942,942
9,942,942,942,942,942,942,942,942


In [35]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [45]:
X = train_df[['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday']]
y = train_df['Sales']

In [44]:
X['StateHoliday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [58]:
zz = pd.get_dummies(train_df['StateHoliday'], prefix='state_holiday')
state_holiday = zz[['state_holiday_a', 'state_holiday_b', 'state_holiday_c']]
X = pd.merge(X, state_holiday, how='inner', left_index=True, right_index=True)

In [60]:
X

,DayOfWeek,Open,Promo,SchoolHoliday,state_holiday_a,state_holiday_b,state_holiday_c
0,5,1,1,1,0,0,0
1,5,1,1,1,0,0,0
2,5,1,1,1,0,0,0
3,5,1,1,1,0,0,0
4,5,1,1,1,0,0,0
5,5,1,1,1,0,0,0
6,5,1,1,1,0,0,0
7,5,1,1,1,0,0,0
8,5,1,1,1,0,0,0
9,5,1,1,1,0,0,0


In [59]:
# Train a random forest on only 
from sklearn.model_selection import cross_val_score, KFold
rf = RandomForestRegressor(n_jobs=-1)
scores = cross_val_score(rf, X, y, cv=KFold(n_splits=5, shuffle=True), scoring='neg_mean_squared_error', n_jobs=-1)
scores            

array([-6630029.47408732, -6547094.80655806, -6615503.85446302,
       -6601047.18524292, -6516283.55160603])